In [3]:
import numpy as np
wordsList = np.load('/Users/xintongchen/LSTM-Sentiment-Analysis/training_data/wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('/Users/xintongchen/LSTM-Sentiment-Analysis/training_data/wordVectors.npy')
print ('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


In [4]:
print(len(wordsList))
print(wordVectors.shape)

400000
(400000, 50)


In [4]:
oxfordIndex = wordsList.index('oxford')
wordVectors[oxfordIndex]

array([-0.92737 ,  0.92025 , -0.9622  , -1.5487  , -0.28903 , -0.46466 ,
       -1.0001  , -0.76671 , -0.061258, -0.51013 , -0.17205 ,  0.61229 ,
       -0.23128 , -1.2463  , -0.64682 ,  0.068583, -0.67415 ,  0.30757 ,
       -0.99604 ,  0.63783 ,  1.0153  ,  0.8797  ,  0.17044 ,  0.40966 ,
        0.11479 , -1.3514  , -0.11413 , -0.57953 , -1.5189  ,  0.19853 ,
        1.4502  , -0.43875 ,  0.11869 , -0.64258 ,  0.59234 , -0.3434  ,
        0.11201 ,  0.62576 ,  0.3458  , -0.42978 ,  0.70889 , -0.78247 ,
       -0.44106 ,  1.0735  , -0.49797 ,  0.7095  ,  0.56106 , -0.64983 ,
        0.14423 , -0.76011 ], dtype=float32)

In [5]:
import tensorflow as tf

#maxSeqLength = 10 #Maximum length of sentence
#numDimensions = 300 #Dimensions for each word vector

firstSentence = np.zeros((maxSeqLength), dtype='int32')

firstSentence[0] = wordsList.index("i")

firstSentence

In [6]:
numDimensions = 300
maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

In [7]:
tf.__version__

'1.0.0'

In [8]:
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [10]:
import os
#os.chdir('/Users/xintongchen/LSTM-Sentiment-Analysis')
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('/Users/xintongchen/LSTM-Sentiment-Analysis/models'))

In [11]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

clean text

In [12]:
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

Sentiment analysis on reddit comments

In [13]:
inputText = "racle 11i and Discoverer. Manufacturing jack of all trades. Master Scheduling by trade everything from AP and GL to Eng in practice"
inputMatrix = getSentenceMatrix(inputText)


In [14]:
inputMatrix

array([[399999, 362942,      5, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0],
       ...,
       [     0,      0,      0, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0]], dtype=int32)

In [16]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

if (predictedSentiment[0] > predictedSentiment[1]):
    print ("Positive Sentiment")
else:
    print ("Negative Sentiment")

Positive Sentiment


In [17]:
predictedSentiment

array([ 0.6854036 , -0.53787905], dtype=float32)